# Deep Reinforcement Learning aplicado a Super Mario Bros

## Presentación de la práctica

En esta práctica, vais a implementar un agente de RL que aprenda a jugar al videojuego Super Mario Bros en su versión para NES. Para ello, haréis uso de un entorno de Gymnasium como base y lo modificaréis y ampliaréis para mejorarlo. Una vez hechos los cambios, usareis la librería Stable Baselines 3 para llevar a cabo el entrenamiento del agente en dicho entorno. Tenéis más información, detalles y enlaces en el PDF adjunto a este notebook.

![NES Mario](./media/video-1-1.gif)

### Estructura del Notebook

Este Jupyter notebook se distribuye en 4 partes: Configuración del entorno, modificación y ampliación, entrenamiento del agente y evaluación de desempeño. En cada parte, se os proporcionaran instrucciones base de qué debéis hacer, así como código inicial del que partir. 

En cada bloque de código, deberéis rellenar aquellas partes marcadas con un 'TODO:'. Además, también podéis hacer más de lo que se os pide. Tened en cuenta que, como se indica en el enunciado de la práctica, **se busca que investiguéis, entendais el problema que estais afrontando, exploréis formas de mejorar el entrenamiento, y en definitiva que trabajeis vuestra creatividad y resolutividad**. 

Sentíos libres de añadir nuevas celdas de código, celdas de texto para indicar qué estáis haciendo y facilitar la corrección, etc.

### Configuración inicial del Notebook

Los jupyter notebooks son códigos interactivos divididos por bloques de ejecución. Para ejecutarse, debemos indicar al notebook el kernel de ejecución con el que debe funcionar. En nuestro caso, este debe ser el entorno de conda o equivalente con Python 3.8 y las dependencias indicadas en el archivo de requisitos que hayamos creado. [Aquí](https://code.visualstudio.com/docs/datascience/jupyter-kernel-management#:~:text=Go%20to%20the%20Command%20Palette,notebook%2C%20select%20Connect%20to%20Codespace.) tenéis un ejemplo de como hacerlo en VSCode.

Tened en cuenta que las celdas se ejecutan secuencialmente, y se genera una tabla de variables conforme se van ejecutando las celdas. Esto quiere decir que si quiero usar una variable que he definido en una celda previa, debo asegurarme que dicha celda se ha ejecutado previamente. Así mismo, si modifico algo en una celda anterior tras haber ejecutado las siguientes, debo volver a ejecutar la celda anterior para ver reflejados los cambios.

### Entrega final

Como se indica en el enunciado de la práctica, tendréis que entregar una memoria en PDF de todo el proceso, el modelo del agente entrenado que consideréis mejor, y este mismo Notebook completo con vuestros cambios. Es importante que, a la hora de entregar el notebook, os fijéis que en la salida de cada celda se reflejan las últimas ejecuciones que hayáis hecho. De ese modo, cuando vayamos a corregir vuestro código, veremos el resultado de vuestra última ejecución sin necesidad de ejecutarlo nosotros mismos (útil para evaluar los logs del proceso de aprendizaje, por ejemplo).

Dicho esto, **¡Vamos allá!**

## Configuración inicial del entorno

Lo primero que debemos hacer es importar las librerías que necesitamos para hacer funcionar nuestro entorno. Debemos importar la librería Gymnasium, así como el emulador NES Py y el propio entorno de Super Mario:

In [3]:
import gymnasium as gym
# Gymnasium ofrece diferentes tipos de espacios para definir el espacio de acción y el espacio de observación.
# En este caso, usamos Box para definir un espacio de observación continuo, y Discrete para un espacio de acción discreto.
from gymnasium.spaces import Box, Discrete

# Para modificar el entorno de Gymnasium, usamos la clase Wrapper.
# Wrapper es una clase base que permite modificar el comportamiento de un entorno sin cambiar su implementación.
from gymnasium import Wrapper

# Del emulador de NES, importamos el espacio de acción que emula el joystick.
from nes_py.wrappers import JoypadSpace

# Importamos el entorno de Super Mario Bros.
import gym_super_mario_bros
# Como indica la documentacion, el entorno de Super Mario Bros. tiene diferentes espacios de acción.
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY

A continuación, siguiendo la [documentación](https://github.com/Kautenja/gym-super-mario-bros) del entorno base de Super Mario, vamos a implementar un bucle de test del entorno para visualizar si funciona correctamente. 

**Importante**: Tened en cuenta que el entorno original fue implementado con Gym<0.21, que tenía una API ligeramente distinta. Esto puede crearnos problemas de compatibilidad con lo que devuelven las funciones `step()` y `reset()`. Investigad sobre este cambio y sobre como solucionarlo fácilmente (pista: observad los parámetros de `gym.make()`)

In [ ]:
# TODO: Inicializa el entorno de Super Mario Bros. Inicializa el nivel 1 del mundo 1
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', apply_api_compatibility=True, render_mode="human")

# Una vez inicializado el entorno, debemos envolverlo en el wrapper del Joystick de NES para poder elegir las acciones.
action_type = RIGHT_ONLY  # Puede ser SIMPLE_MOVEMENT, COMPLEX_MOVEMENT o RIGHT_ONLY. Vamos a usar RIGHT_ONLY para este ejemplo.
env = JoypadSpace(env, action_type)

# TODO: Muestra cuales son las dimensiones del espacio de observación y el espacio de acción.
obs_dim = env.observation_space.shape
action_dim = env.action_space.n
print("Observation space dimension:", obs_dim)
print("Action space dimension:", action_dim)

### BUCLE DE TEST ###

# En primer lugar, inicializamos el entorno.
obs, info = env.reset()

# Vamos a hacer un bucle infinito para que el agente juegue al juego tomando acciones aleatorias.
# Cuando queramos parar, clickamos en el STOP de la celda de Jupyter. (de ahi el uso de la excepcion KeyboardInterrupt)
try:
    while True:
        # TODO: Toma una acción aleatoria del espacio de acción.
        action = env.action_space.sample()  # Random action
        # TODO: Ejecuta la acción en el entorno y obtiene el nuevo estado, la recompensa, si ha terminado el episodio y la información adicional.
        obs, reward, done, trunk, info = env.step(action)
        # TODO: Renderiza el entorno para visualizar el juego.
        env.render()
        # Si el episodio ha terminado o si el entorno ha sido reiniciado, reiniciamos el entorno.
        if done or trunk:
            obs, info = env.reset()
            print("Episode finished. Resetting environment.")

except KeyboardInterrupt:
    print("Exiting...")
    env.close()

¡Enhorabuena! Con esto, habreis implementado un bucle de ejecución típico de un problema de RL, y habréis visualizado a Mario moviendose por su mundo tomando decisiones aleatorias dentro del espacio de acciones que le hemos indicado. 

Sentíos libres de experimentar con algunos de los parametros y ver cómo varia la ejecución (por ejemplo, los diferentes espacios de acciones). Investigad también qué es exactamente la observacion (es una imagen? la puedo visualizar con OpenCV o similares?) y la recompensa.

## Modificación y ampliación del entorno

Como habeis visto explorando el entorno y leyendo la documentación de este, por defecto la observación que recibimos es una imagen de color RGB con dimensiones 256x240, y el valor de recompensa se calcula asumiendo que el objetivo es avanzar a la derecha lo más rápido posible sin morir. 

Sin embargo, usar directamente esta información en crudo puede no ser lo mejor de cara al entrenamiento de nuestro agente. Por ello, vamos a aplicar algún preprocesado a la información del entorno para optimizar el rendimiento y tratar de proporcionar una señal de aprendizaje mejor. 

Así pues, en las siguientes celdas vamos a implementar este preprocesado. **Importante**: además del procesado que sugerimos aquí, sentíos libres de modificar la observación proporcionada y de experimentar con todos los datos que tenéis para modificar la función de recompensa. Estos datos se incluyen en el diccionario "info" que se devuelve junto a la observacion y la recompensa tras cada "step" del entorno.

### Preprocesado de la imagen

En este punto, estamos trabajando con los pixels en crudo del juego tal y como los presenta el emulador de NES. Sin embargo, este nivel de detalle puede ser más de la que en realidad necesita nuestro agente para entrenar, y por tanto estamos introduciendo información innecesaria que puede distraer a nuestro agente durante el aprendizaje, y que seguro va a hacer cada bucle de aprendizaje más costoso computacionalmente.

Así pues, parad a pensar un momento: ¿Necesitamos saber el color del cielo o las tuberias? ¿Podría servirnos una imagen mas pequeña? ¿Qué valores puede tomar cada entrada de la red (pixel de la imagen)? ¿Puedo reducir la varianza entre los valores para estabilizar el entrenamiento?.

Todo esto es lo que vais a enfrentar ahora. Actualmente, tenemos la imagen RGB de tamaño 256x240. Vamos a convertirla a escala de grises y reducir su tamaño para reducir la dimensionalidad de nuestra observación y así acelerar el tiempo de computo, eliminando información no relevante de la imagen. Además, es útil normalizar los valores de nuestra imagen, de modo que reducimos el rango de valores a [0,1].

A continuación, implementad una función que tome como entrada la observación en crudo del entorno (la imagen), y que devuelva la imagen con el procesamiento descrito anteriormente. Para la redimensión, convertid la imagen en tamaño 84x84 para empezar, pero sentios libres de explorar otros tamaños si creeis que es demasiado grande/pequeña. Podéis usar OpenCV para el tratamiento de la imagen.

In [5]:
import cv2
import numpy as np

# Dimensiones de la imagen procesada
HEIGHT = 84
WIDTH = 84

# Aplica un procesamiento a la imagen del entorno para que sea más fácil de manejar.
def process_frame(frame):    
    if frame is not None:        
        #TODO: Convierte el frame a escala de grises
        if frame.ndim == 2:
            gray = frame
        else:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        #TODO: Redimensiona la imagen
        resized = cv2.resize(gray, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
        #TODO: Normaliza la imagen dividiendo por 255.0
        normalized = resized / 255.0

        return normalized.astype(np.float32)  # Ensure the output is float32
    else:
        return np.zeros((HEIGHT, WIDTH), dtype=np.float32)

Con esto, pasamos de tener una observación de dimensiones 3x256x240, a 1xHEIGHTXWIDHT, reduciendo drásticamente nuestra representación del estado del mundo de Mario. 

### Personalización de la función de recompensa

Si consultais la documentación del entorno, teneis una descripción clara de como se define la funcion de recompensa base para el entorno. Se asume que el objetivo del juego es moverse tan a la derecha como se pueda, en el menor tiempo posible, sin morir. Por tanto, la recompensa es una suma de diferentes valores que codifican estos objetivos. 

Sin embargo, contamos con información adicional sobre el juego que puede hacer que Mario tenga más guía con respecto a su objetivo en el mundo. ¿Es importante coger monedas? ¿Nos interesa maximizar la puntuación dentro del juego? ¿Es positivo recoger modificadores como champiñones o flores? ¿Qué pasa cuando cojo la bandera?.

Toda esta información está disponible en el diccionario `info` devuelto por la función `step()`. Así pues, a continuación vamos a ampliar la función de recompensa para incluir parte de esta información. Vamos a proponer algún cambio concreto, pero **sentíos libres de personalizar la función de recompensa tanto como querais**, explicando la lógica detrás de cada cambio.

Para hacer estas modificaciones, vamos a implementar un "Wrapper" sobre el entorno. Los Wrappers o "envoltorios" son capas que se colocan sobre el entorno base, y que nos permiten editarlo sin tener que rehacer los métodos por completo. El concepto es parecido al de herencia de clases que ya conocéis. Podéis consultar la [documentación](https://gymnasium.farama.org/api/wrappers/) de Gymnasium al respecto.

In [6]:
# Creamos una clase que hereda de Wrapper para modificar el entorno.
# Esta clase se encargará de procesar la imagen y modificar la recompensa.
class CustomReward(Wrapper):
    def __init__(self, env=None, w_kill=1.0):
        super(CustomReward, self).__init__(env)
        # Actualizamos el espacio de observación y el espacio de acción.
        self.observation_space = Box(0.0, 1.0, shape=(HEIGHT, WIDTH), dtype=np.float32)
        self.action_space = Discrete(env.action_space.n)
        # Inicializamos la información adicional.
        self.info = {}
        # Inicializamos la posición x inicial de Mario.
        self.current_x = 40

        #TODO: Inicializad las variables necesarias para vuestra implementación.

        self.prev_coins  = 0            # monedas recogidas en el paso anterior
        self.w_kill = w_kill
        self.flag_rewarded = False

    def step(self, action):
        ### Modificamos la función step() para procesar la imagen y modificar la recompensa. ###

        # Ejecutamos la acción en el entorno y obtenemos el nuevo estado, la recompensa, si ha terminado el episodio y la información adicional.
        obs, reward, done, truncated, info = self.env.step(action)
        self.info = info
        #TODO: Aplicamos el procesamiento a la imagen.
        obs = process_frame(obs)

        #TODO: Personalizamos la recompensa.
        #TODO: Vamos a añadir una recompensa terminal positiva si Mario llega a la meta y una negativa si no
        if info.get("flag_get", False) and not self.flag_rewarded:
            reward += 50.0
            self.flag_rewarded = True
            print("🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩")
        elif done and not info.get("flag_get", False):
            reward -= 20.0

        # 2. Monedas
        coin_diff = info["coins"] - self.prev_coins
        reward += 1 * coin_diff
        self.prev_coins = info["coins"]

        # + Recompensa por muertes de enemigos
        reward += 0.05 * (info["score"] - self.prev_score)
        self.prev_score = info["score"]

        # 5. Potential-based shaping
        reward += 0.1 * (info["x_pos"] - self.last_x)
        self.last_x = info["x_pos"]

        return obs, reward / 10., done, truncated, info

    def reset(self, **kwargs):
        ### Modificamos la función reset() para procesar la imagen y reiniciar las variables necesarias. ###
        #TODO: Reiniciad las variables necesarias para vuestra implementación.
        kwargs.pop('seed', None)

        # Reiniciamos el entorno y obtenemos el estado inicial procesado y la información adicional.
        obs, info = self.env.reset(**kwargs)

        # AÑADIDAS
        self.prev_coins    = 0
        self.flag_rewarded = False
        self.prev_score    = info.get("score", 0)
        self.last_x     = info.get("x_pos", 0)

        return process_frame(obs), info

Llegados a este punto, hemos aplicado un procesado a la imagen en crudo obtenida por el entorno, y hemos personalizado la función de recompensa para añadirle más expresividad y así ayudar a Mario en su aprendizaje. Aseguraos de inicializar y reiniciar las variables que utiliceis para vuestra implementación de recompensa en los métodos `__init__` y `reset`.



### Obtención de información temporal.

Hay un problema que no hemos enfrentado todavía de cara a poder plantear un entrenamiento de un agente de RL para que juegue a Mario: ¿Qué pasa con la información temporal que los humanos manejamos de forma implicita?

Por ejemplo: Si observamos un único frame en el que Mario está encima de un precipicio, ¿Está cayendo o está saltando al otro lado? ¿El caparazón viene hacia nosotros o se está alejando?. Esta información nosotros la procesamos muy fácilmente de forma intuitiva, pero nuestro agente solo está observando un frame cada vez, por lo que le es imposible deducir esta información. 

Entonces, ¿como manejamos la temporalidad? Hay arquitecturas de redes neuronales que manejan estados internos (LSTM, GRU), pero nosotros vamos a abordar el problema antes de llegar a la creación del agente, con un método sencillo pero a la vez eficaz: En lugar de observar solo un frame, vamos a observar una secuencia de N frames de forma simultanea.

Para ello, vamos a implementar otro Wrapper adicional, en el cual definiremos un número N de frames deseados, y los "apilaremos". De este modo, la observación del agente pasará a ser de la forma N x HEIGHT x WIDTH, dotándole así de información temporal con la que deducir el movimiento de los objetos y del propio Mario.

**Importante**: Tened cuidado con las dimensionalidades de las variables que manejais. Además, hay algo que debemos tener en cuenta: Si vamos a tratar una secuencia de frames como una única observación, ¿Qué pasa con la recompensa? Tened esto presente a la hora de implementar vuestro código.

In [7]:
from collections import deque
import numpy as np

# Wrapper para apilar frames del entorno.
# Esta clase se encargará de apilar los frames del entorno para crear un stack de frames.
# Esto es útil para que el agente pueda ver el movimiento de Mario y aprender a jugar mejor.
N_FRAMES = 4
class CustomStackFrames(Wrapper):
    def __init__(self, env, n_frames=N_FRAMES):
        super(CustomStackFrames, self).__init__(env)
        self._n_frames = n_frames
        # TODO: Actualiza el espacio de observaciones del entorno para que tenga en cuenta el número de frames apilados.
        
        self.observation_space = Box(
            low=0, high=1.0,
            shape=(self._n_frames, HEIGHT, WIDTH),
            dtype=np.float32
        )

        # TODO: Inicializa el stack de frames.
        self.frames = deque(maxlen=self._n_frames)

    def step(self, action):
        #TODO: Modifica la función step() para apilar los frames. Piensa qué hacer con la recompensa, y como manejar el final del episodio.
        '''done = False
        truncated = False'''

        # Ejecuta la acción en el entorno original
        obs_raw, reward, done, truncated, info = self.env.step(action)

        # Procesamos el nuevo frame y lo añadimos al stack
        frame = process_frame(obs_raw)
        self.frames.append(frame)

        # Construimos la observación apilada
        stacked_obs = np.stack(self.frames, axis=0)

        # Si el episodio termina o se trunca, limpiamos el stack para el próximo reset
        if done or truncated:
            self.frames.clear()

        # TODO: Acuerdate de devolver lo que devuelve la función step() original!

        # Devolvemos la observación apilada, recompensa, flags y info
        return stacked_obs, reward, done, truncated, info

    def reset(self, **kwargs):
        # TODO: Modifica la función reset() para apilar los frames.
        # En este caso, considera que el stack de frames es el estado inicial repetido n veces.
        
        # Descartamos el seed (DummyVecEnv/VecEnv se lo pasa, pero el env legacy no lo acepta)
        kwargs.pop('seed', None)

        # Reiniciamos el entorno original
        obs_raw, info = self.env.reset(**kwargs)

        # Creamos el frame procesado y llenamos el stack con N copias
        frame = process_frame(obs_raw)
        self.frames = deque([frame] * self._n_frames, maxlen=self._n_frames)

        # Devolvemos la observación apilada inicial y la info
        return np.stack(self.frames, axis=0), info
    
########################## Nuevo WRAPPER ##########################

class ActionRepeat(Wrapper):
    """
    Wrapper que repite la misma acción durante `repeat` pasos consecutivos.
    - Acumula la recompensa.
    - Si `done` o `truncated` ocurren, deja de repetir y retorna inmediatamente.
    """
    def __init__(self, env, repeat: int = 4):
        super().__init__(env)
        self.repeat = repeat
        # Conservamos los espacios originales
        self.observation_space = env.observation_space
        self.action_space = env.action_space

    def step(self, action):
        total_reward = 0.0
        done = False
        truncated = False
        info = {}
        for _ in range(self.repeat):
            obs, reward, done, truncated, info = self.env.step(action)
            total_reward += reward
            if done or truncated:
                break
        return obs, total_reward, done, truncated, info

    def reset(self, **kwargs):
        # Solo delegamos al reset original
        return self.env.reset(**kwargs)

Con esto, vamos a concluir la fase de modificación del entorno base de Super Mario. Llegados a este punto, como mínimo debes haber:

- Implementado el preprocesado a la imagen para convertirla en escala de gris y reducir su dimensionalidad.
- Personalizado tu recompensa para incluir una bonificación por completar el nivel.
- Implementado un wrapper para apilar un conjunto de frames consecutivos y así dotar de información temporal al agente.

Además de esto, puedes haber implementado cualquier otra modificación al entorno que creas conveniente, especialmente a la función de recompensa. 

Para terminar, vamos a juntarlo todo para aplicar los wrappers con nuestros cambios al entorno. Para ello, vamos a implementar una función que nos creará un entorno de Mario del mundo y nivel escogidos, con el tipo de acción especificado, y aplicando los wrappers que hemos implementado:

In [8]:
def create_mario_env(world, stage, action_type, n_frames_repeat, n_frames_stack, render_mode):    
    #TODO: Crea el entorno base de Super Mario Bros. con el mundo y el nivel especificados.
    env = gym_super_mario_bros.make(
        f"SuperMarioBros-{world}-{stage}-v0",
        render_mode=render_mode,
        apply_api_compatibility=True
    )

    # Envuelve el entorno en el wrapper del Joystick de NES para poder elegir las acciones.
    env = JoypadSpace(env, action_type)
    env = ActionRepeat(env, repeat=n_frames_repeat)
    #TODO: Envuelve el entorno en los wrappers de CustomStackFrames y CustomReward.
    env = CustomReward(env)
    env = CustomStackFrames(env, n_frames_stack)

    return env

Finalmente, comprobad que los wrappers se están aplicando correctamente. Para ello, cread un bucle similar al del inicio de la práctica, y analizad su funcionamiento:

In [9]:
# Variables de configuración
# Puedes cambiar el mundo y el nivel para probar diferentes niveles de Super Mario Bros.
# También puedes cambiar el tipo de acción y el número de frames apilados.
WORLD = 1
STAGE = 1
ACTION_TYPE = SIMPLE_MOVEMENT
N_FRAMES_STACK = 4
N_FRAMES_REPEAT = 4
CHECK_FREQ = 10_000

In [ ]:
#TODO: Crea el entorno de Super Mario Bros. con el mundo y el nivel especificados.
env = create_mario_env(WORLD, STAGE, action_type=ACTION_TYPE, n_frames_repeat=N_FRAMES_REPEAT, n_frames_stack=N_FRAMES_STACK, render_mode="rgb_array")

# Reinicia el entorno y obtiene el estado inicial procesado y la información adicional.
obs, info = env.reset()

# Bucle de prueba para analizar el entorno. Personalizadlo para leer la información que queráis.
# Como ejemplo, vamos a imprimir la forma de la observación.
try:
    while True:
        action = env.action_space.sample()  # Random action
        obs, reward, done, trunk, info = env.step(action) # Ejecuta la acción en el entorno y obtiene el nuevo estado, la recompensa, si ha terminado el episodio y la información adicional.
        print("Observation shape:", obs.shape)
        env.render()  # Render the environment
        if done or trunk:
            obs = env.reset()
            print("Episode finished. Resetting environment.")

except KeyboardInterrupt:
    print("Exiting...")
    env.close()

## Entrenamiento del agente con Stable Baselines 3

Ya tenemos personalizado nuestro entorno de Super Mario con una observación modificada y una función de recompensa extendida, y hemos verificado que los cambios implementados en nuestros wrappers se aplican correctamente al entorno.

¡Pues a entrenar! Vamos a empezar a configurar el proceso de entrenamiento usando la librería Stable Baselines 3. Teneis una documentación extensiva en su [web](https://stable-baselines3.readthedocs.io/en/master/index.html).

SB3 sigue la API de Gymnasium en la implementación de algoritmos de RL, además de proporcionar utilidades como métodos de evaluación de políticas, monitorización usando Tensorboard, personalización de callbacks durante el entrenamiento, implementación de arquitecturas personalizadas de redes usando PyTorch, o paralelización de entornos en CPU para acelerar el entrenamiento si el hardware lo permite.

Así pues, en esta parte de la práctica vais a escoger el algoritmo de RL para vuestro entrenamiento, el tipo de arquitectura de red neuronal que parametrizará al agente, y los distintos parámetros que tiene el proceso de aprendizaje. 

**Importante**: Recordad justificar las decisiones de diseño que tomáis dada la tarea a resolver: ¿Por qué escoges un algoritmo u otro? ¿Por qué un MLP o una CNN? ¿Implementas tu propia arquitecura? Explicad todo lo mejor que podáis, ya que una toma de decisiones crítica e informada es uno de los objetivos transversales de esta práctica.

### Implementación de entornos vectorizados

SB3 permite la vectorización de entornos en CPU. Esto se refiere a ejecutar diversos entornos de forma simultánea para obtener más datos de entrenamiento con los que ajustar nuestra política. Dentro de los tipos de vectorizado disponibles, tenemos:

- DummyVecEnv: Inicializa varios entornos simultaneos, pero los ejecuta secuencialmente en CPU. No ganamos en paralelización, pero para entornos sencillos suele ser mejor que usar multiprocesado
- SubprocVecEnv: Aprovecha la capacidad de paralelización de las CPU multicore para ejecutar entornos en cores diferentes, lo que puede llegar a aumentar la velocidad de entrenamiento. 

Los algoritmos de aprendizaje de SB3 esperan que el entorno sea vectorizado usando wrappers de algunas de estas clases, aunque solo sea 1 único entorno. Por ello, vamos a implementar la vectorización de entornos para nuestro entorno de Mario. Después, experimentad y decidid si en vuestro caso es necesario aumentar el número de entornos paralelos o no, y si los ejecutareis secuencialmente o con threading con SubprocVecEnv.

In [ ]:
# Primero, importamos las librerías necesarias para el vectorizado de entornos
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

# Vamos a utilizar la función make_vec_env para crear un entorno vectorizado.
# Esta función crea un entorno vectorizado utilizando el entorno base que hemos creado anteriormente.
# El número de entornos vectorizados por defect es 4, pero cambiadlo en base a vuestra CPU y analisis de rendimiento.
NUM_ENVS = 16

env = make_vec_env(
    lambda: create_mario_env(WORLD, STAGE, action_type=ACTION_TYPE, n_frames_repeat=N_FRAMES_REPEAT, n_frames_stack=N_FRAMES_STACK, render_mode="rgb_array"),
    n_envs=NUM_ENVS,
    vec_env_cls=SubprocVecEnv, # Puede ser DummyVecEnv o SubprocVecEnv. Cambiadlo en base a vuestra CPU y analisis de rendimiento.
    monitor_dir="./mario_monitor_dir/",
    seed=33,
)
# Al crear el entorno vectorizado, os saldran unos warnings. No os preocupeis, son normales.
print("Vectorized environment created.")
print("Number of environments:", env.num_envs)

/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
/home/ferrer/miniconda3/envs/mario_e

Vectorized environment created.
Number of environments: 16


### Creación de callbacks durante el entrenamiento.

Como ya sabéis, un callback es una función que se llama automáticamente cuando sucede algún evento. En este caso, podemos implementar callbacks para el proceso de entrenamiento de modo que nos guarden puntos intermedios de este. Esto es útil en el caso de que tengamos que parar el entrenamiento a mitad de ejecución, o que la mejor política no sea la última. 

Por ello, vamos a implementar un callback que nos guarde checkpoints intermedios de nuestro entrenamiento.

Además, podéis implementar otro tipo de callbacks como alguno que modifique el learning rate durante el aprendizaje, por ejemplo. De nuevo, tenéis libertad en explorar estas opciones.

In [ ]:

from stable_baselines3.common.callbacks import BaseCallback
import os

# Callback para guardar el modelo y registrar la información durante el entrenamiento.
# Este callback se ejecuta cada cierto número de pasos y guarda el modelo en la ruta especificada.
class TrainAndLogCallback(BaseCallback):
    def __init__(self, name, check_freq, save_path, start_steps=0, verbose=1):
        super(TrainAndLogCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        self.start_steps = start_steps
        self.name = name

    def _init_callback(self):
        # Creamos la carpeta de guardado si no existe.
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        # Guardamos el modelo cada check_freq pasos.
        if self.n_calls % self.check_freq == 0:
            if self.save_path is not None:
                # convierto a pasos reales
                real_steps = self.n_calls * self.model.n_envs
                filename   = os.path.join(self.save_path, f"model_{real_steps}_{self.name}")
                self.model.save(filename)
        return True

#TODO: (Opcional) Implementad otros callbacks si lo considerais necesario.

from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CallbackList

# 2) Callback de parada temprana al alcanzar un umbral de recompensa media
stop_callback = StopTrainingOnRewardThreshold(
    reward_threshold=350,    # adapta este umbral a vuestra métrica
    verbose=1
)

# 1) Callback de evaluación periódica en un entorno aparte
eval_env = create_mario_env(WORLD, STAGE, action_type=ACTION_TYPE, n_frames_repeat=N_FRAMES_REPEAT, n_frames_stack=N_FRAMES_STACK, render_mode="rgb_array")
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./best_model/",
    log_path="./eval_logs/",
    eval_freq=10_000/NUM_ENVS,          
    n_eval_episodes=10,
    deterministic=True,
    render=True,
    callback_on_new_best=stop_callback
)

if ACTION_TYPE == SIMPLE_MOVEMENT:
    run_id = f"w{WORLD}_s{STAGE}_r{N_FRAMES_REPEAT}_ac-SIMPLE_MOVEMENT-"
elif ACTION_TYPE == RIGHT_ONLY:
        run_id = f"w{WORLD}_s{STAGE}_r{N_FRAMES_REPEAT}_ac-RIGHT_ONLY-" 

# 3) Lista de callbacks para pasarlos luego al método .learn()
save_path = os.path.join("./mario_models", run_id)

callback = CallbackList([ 
    TrainAndLogCallback(name=run_id, check_freq=CHECK_FREQ//NUM_ENVS, save_path=save_path, start_steps=0, verbose=1),
    eval_callback, 
    stop_callback, 
    ])


### Entrenamiento de la política

En este punto, ya estamos en posición de implementar el entrenamiento de nuestro agente. SB3 tiene una forma muy sencilla de iniciar un entrenamiento, revisad la documentación y lo encontraréis rápido.

Ahora, tenéis que escoger el algoritmo y la arquitectura que vais a utilizar. Revisad la lista de algoritmos disponibles en SB3 y sus características, y meditad cual podría ser una opción que encajase con nuestro entorno de Mario. **No hay una única respuesta correcta**, si bien es cierto que, por las características del entorno, podréis excluir algunos algoritmos directamente. 

En cuanto a la arquitectura, SB3 permite escoger arquitecturas base de forma sencilla, así como implementar vuestra propiar arquitectura en PyTorch heredando de alguna de sus clases base. Revisad la forma de especificar la arquitectura y tomad la decisión que creais correcta para la tarea que enfrentáis: ¿Qué tipo de datos va a procesar mi red? ¿Que arquitecturas son adecuadas para estos datos? ¿Tengo alguna arquitectura por defecto que me proporcione lo que necesito, o debo implementarla?.

Por último, hay diversos parámetros de aprendizaje que se pueden ajustar: Learning rate, número de steps por epoca de aprendizaje, steps totales a ejecutar durante el entrenamiento... Analizad la tarea e iterad para ajustar estos parámetros, si bien es cierto que cada algoritmo de SB3 viene con un set de parámetros por defecto que suele ser decente para muchas tareas.

Una vez hayáis estudiado la documentación y los ejemplos, implementad el entrenamiento en la siguiente celda:

In [26]:
#TODO: Inicializa el modelo con el algoritmo deseado (PPO, DQN, A2C, etc.), el entorno vectorizado y los parámetros deseados. 
# Aseguraos de indicar verbose=1 para que se vea el progreso del entrenamiento.

from stable_baselines3 import PPO
import os

log_path = os.path.join("./train", run_id)

model = PPO(
    "CnnPolicy",           # arquitectura CNN para procesar los stacks de frames
    env,                    # entorno vectorizado creado arriba
    learning_rate=2.5e-4,   # LR típico para PPO
    n_steps=128,            # número de timesteps por rollout
    batch_size=64,          # tamaño de minibatch
    n_epochs=8,             # pasadas de SGD por rollout
    gamma=0.9,             # factor de descuento
    gae_lambda=0.98,
    clip_range=0.2,
    ent_coef=0.01,
    max_grad_norm=0.5,
    verbose=1,
    tensorboard_log=log_path,
    policy_kwargs={'normalize_images': False},
    device='cuda:0'          # <- Le dices que use la GPU 0
)

# TODO: Entrena el modelo con el número de pasos deseado
# Para un entrenamiento rápido pero efectivo, vamos a usar 1e6 pasos. (approx. 1h en una CPU i7 segun el vectorizado que useis)
# Este número puede ser ajustado en base a la velocidad de entrenamiento y el rendimiento del modelo.
total_timesteps = 250_000*NUM_ENVS
model.learn(total_timesteps=total_timesteps, callback=callback)
# Código de entrenamiento aquí!

# Guardamos el modelo final.
#model.save("mario_final_model")
# Cerramos el entorno.
env.close()

Using cuda:0 device
Logging to ./train/w1_s1_r4_ac-SIMPLE_MOVEMENT-/PPO_1


/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/stable_baselines3/common/callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x70fd3c0638e0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x70fd3c072520>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/ferrer/miniconda3/envs/mario_env/lib/p

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 44.4     |
|    ep_rew_mean     | 24       |
| time/              |          |
|    fps             | 1640     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 95.7        |
|    ep_rew_mean          | 40.2        |
| time/                   |             |
|    fps                  | 1291        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014033088 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | 1.66e-05    |
|    learning_rate        | 0.

/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Eval num_timesteps=10000, episode_reward=0.74 +/- 0.00
Episode length: 2005.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 2.00e+03    |
|    mean_reward          | 0.74        |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008588571 |
|    clip_fraction        | 0.091       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.9        |
|    explained_variance   | 0.325       |
|    learning_rate        | 0.00025     |
|    loss                 | 2.61        |
|    n_updates            | 32          |
|    policy_gradient_loss | -0.00734    |
|    value_loss           | 6.28        |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 181      |
|    ep_rew_mean     | 61.5     

¡Enhorabuena! Ya habéis entrenado vuestra política para jugar a Super Mario. Durante el entrenamiento, habréis ido viendo logs del proceso de aprendizaje que SB3 muestra por defecto. con métricas como:

- ep_len_mean: Duración media de los episodios en la época.
- ep_rew_mean: Recompensa media del agente en esa época.
- fps: Frames por segundo procesados. A mayor FPS, mayor velocidad de ejecución. Usad esta métrica para elegir el tipo de vectorizado y el numero de entornos.

Hay otras métricas mostradas durante el entreno. Revisad la documentación para entenderlas y usad esta información para refinar vuestro entrenamiento. Porque efectivamente, es muy probable que en este primer intento el resultado no sea el mejor. 

Para comprobarlo, vamos a evaluar nuestra política de forma cuantitativa y cualitativa.

## Evaluación de la política

Una vez completado el entrenamiento de nuestro agente, es hora de evaluar su desempeño. Esto lo podemos hacer de forma cuantitativa y cualitativa. 

En primer lugar, vamos a evaluar cuantitativamente la recompensa media obtenida por el agente en X episodios de ejecución. De este modo, cuando tengamos varios agentes entrenados, podemos compararlos de forma cuantitativa y ver cual es el que obtiene mayor recompensa. Esto puede ser útil para escoger el algoritmo de aprendizaje, por ejemplo. 

Tras esto, evaluaremos de forma cualitativa nuestro agente. ¿Como haremos eso? ¡Viéndole jugar! Vamos a implementar el mismo bucle de test que hicimos al inicio de la práctica, pero esta vez en lugar de tomar acciones aleatorias, tomaremos aquellas que nos indique nuestra política. De ese modo, podremos revisar el comportamiento de nuestro agente y comprobar por qué no funciona, para así tomar decisiones informadas en cuanto a los cambios a implementar para mejorar el desempeño del agente (modificar función de recompensa, parámetros de aprendizaje, etc.)

In [12]:
# SB3 proporciona una función para evaluar el rendimiento del agente entrenado.
# Esta función evalúa el rendimiento del agente en el entorno especificado y devuelve la recompensa media y la desviación estándar.
from stable_baselines3.common.evaluation import evaluate_policy
import time

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

######### EVALUACION CUANTITATIVA #########
# TODO: Creamos un nuevo entorno para evaluar el agente entrenado.
# Este entorno es el mismo que el utilizado para entrenar el agente, pero sin el vectorizado. Vigilad tambien el tipo de renderizado!
env = create_mario_env(
    WORLD, STAGE, action_type=ACTION_TYPE, n_frames_repeat=N_FRAMES_REPEAT, n_frames_stack=N_FRAMES_STACK, render_mode="human"
)
env = DummyVecEnv([lambda: env])

# 2) Entorno “crudo” para renderizar todos los frames
raw_env = gym_super_mario_bros.make(
    f"SuperMarioBros-{WORLD}-{STAGE}-v0",
    render_mode="human",
    apply_api_compatibility=True
)
raw_env = JoypadSpace(raw_env, ACTION_TYPE)

# TODO: Cargamos el modelo entrenado.

model = PPO.load("./best_model/best_model")

#TODO: Evaluamos el rendimiento del agente en el entorno especificado y mostramos la recompensa media y la desviación estándar.
N_EVAL_EPISODES = 1

mean_reward, std_reward = evaluate_policy(
    model, env, n_eval_episodes=N_EVAL_EPISODES, render=False
)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

######### EVALUACION CUALITATIVA #########
# Importante! Los wrappers VecEnv devuelven solo done para el fin de estado, no trunk. 
# Del mismo modo, el reset() devuelve solo el estado y no la info.
# Tenlo en cuenta para el bucle de evaluación.

#TODO: Implementa un bucle de evaluación para analizar el rendimiento del agente entrenado de forma cualitativa.
# Consejo: añade time.sleep(0.02) tras renderizar el entorno para que la velocidad de juego sea más lenta y puedas ver mejor el rendimiento del agente.

# Reset the environment
obs = env.reset()
raw_obs, _ = raw_env.reset()
done = False

try:
    for e in range(N_EVAL_EPISODES):
        while not done:
            # Obten la accion del modelo
            action, _ = model.predict(obs, deterministic=True)
            # Ejecuta la accion en el entorno y observa el nuevo estado, la recompensa, si ha terminado el episodio y la información adicional.
            obs, reward, done, info = env.step(action)
            # Renderiza el entorno para visualizar el juego.
            '''env.render()
            time.sleep(0.2)
            if done:
                print("Episode finished. Resetting environment.")
                obs = env.reset()'''

            # 3.3) En el env “raw” repetimos la misma acción N veces, renderizando cada frame
            for _ in range(N_FRAMES_REPEAT):
                raw_obs, _, raw_done, _, _ = raw_env.step(int(action))
                raw_env.render()         # aquí vemos cada frame
                time.sleep(1 / 60)       # ralentiza a ~60 FPS
                if raw_done:
                    print("Episode finished. Resetting environment.")
                    done = True
                    obs = env.reset()
                    raw_obs, _ = raw_env.reset()
                    break
        
        done = False
except KeyboardInterrupt:
    print("Exiting...")
    env.close()
    raw_env.close()
    exit()
# Close the environment
env.close()
raw_env.close()

/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩
Mean reward: 350.3100116252899 +/- 0.0


/home/ferrer/miniconda3/envs/mario_env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩🚩
Episode finished. Resetting environment.


**¡Enhorabuena!** Con esto, ya has tenido una aproximación real a un problema de RL. Tras completar la práctica, has:

- Evaluado un problema propuesto, y la codificación inicial de este
- Modificado y ampliado la observación y función de recompensa para mejorar el futuro desempeño del agente
- Estudiado los diferentes algoritmos disponibles y escogido el más adecuado para la tarea en base a sus características.
- Iterado sobre varios entrenamientos, analizando métricas de interés y modificando tu entorno y algoritmos de entrenamiento en consecuencia.

Cualquier problema que desees resolver con RL se afronta de un modo muy parecido, así que ya estás preparado para seguir enfrentándote a problemas de tomas de decisiones secuenciales y modelarlos bajo el framework del aprendizaje por refuerzo.

**¡Buen trabajo!**

## ¿Y ahora qué?

Con esto, ya tenéis cubiertos los objetivos de la práctica, no necesitáis hacer más para obtener la máxima nota. 

Pero si os ha picado la curiosidad y queréis seguir "peleando" con este problema, os lanzo las siguientes preguntas:

- **¿Cuantos niveles puede resolver vuestro agente**: El entorno base tiene una opción de randomizar el nivel que se juega en cada reset. Úsalo para entrenar en varios niveles, y no solo en uno.
- **¿Qué espacio de movimiento usais?**: Habéis entrenado con un espacio de acciones restringido, ¡probad dando más libertad al agente!
- **¿Seguro que no podéis diseñar una recompensa más útil?**: Aprovechad vuestro conocimiento del juego para ayudar al agente en su aprendizaje.